In [113]:
# Import necessary Libraries & Paackages
#!pip install zipcodes
#!pip install pgeocode
import requests
from bs4 import BeautifulSoup
import pandas as pd
import zipcodes
import pgeocode
import pymongo

In [114]:
# Ask the User for a Zipcode
zipcode = input('What zipcode do you want the 7 day forcase for? ')

# Test if zipcode is a number
try:
    int(zipcode)
except ValueError as ve:
    raise Exception ('You entered'+zipcode+' as a zipcode, which is not a number.')

# Test if zipcode is real
if zipcodes.is_real(zipcode):
    print(zipcode +' is a correct zipcode')
else:
    raise Exception('Type a correct 5 digit zipcode to continue')

What zipcode do you want the 7 day forcase for? 22932
22932 is a correct zipcode


In [115]:
# Convert Zipcode Latitude & Longitude
nation = pgeocode.Nominatim('us')
query = nation.query_postal_code(zipcode)

latitude = str(query["latitude"])
longitude = str(query["longitude"])

In [116]:
# Scrape National Weather Service for 7-day forcast 
page = requests.get("https://forecast.weather.gov/MapClick.php?lat="+latitude+"&lon="+longitude+"#.Y21NyOzMIUp")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id=="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")

period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]

short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]

In [117]:
# Scrape National Weather Serice for humidity, windspeed, Dewpoint and Last Update Time
page = requests.get("https://forecast.weather.gov/MapClick.php?lat="+latitude+"&lon="+longitude+"#.Y21NyOzMIUp")
soup = BeautifulSoup(page.content, 'html.parser')
current_weather_conditions = soup.find(id="current_conditions_detail").find_all(name = 'td')

humidity = current_weather_conditions[1].string

wind_speed = current_weather_conditions[3].string

dew_point = current_weather_conditions[7].string

last_updated_time = current_weather_conditions[11].string.strip()

In [118]:
# Create 7_day Forecast Dataframe
weather_forecast = pd.DataFrame({
    "Period": periods,
    "Weather": short_descs,
    "Temperature": temps})

#Create Current Conditions Dataframe
current_conditions = pd.DataFrame({
    "Last Updated Time": last_updated_time,
    "Humidity": humidity,
    "Wind Speed": wind_speed,
    "Dew Point": dew_point}, index=[0])

display(weather_forecast)
display(current_conditions)

,Period,Weather,Temperature
0,Overnight,Showers,Low: 58 °F
1,VeteransDay,Heavy Rain,High: 70 °F
2,FridayNight,Showers,Low: 58 °F
3,Saturday,Mostly Sunny,High: 67 °F
4,SaturdayNight,Mostly Cloudy,Low: 40 °F
5,Sunday,Sunny,High: 48 °F
6,SundayNight,Clear,Low: 31 °F
7,Monday,Mostly Sunny,High: 47 °F
8,MondayNight,Partly Cloudy,Low: 31 °F


,Last Updated Time,Humidity,Wind Speed,Dew Point
0,11 Nov 12:15 am EST,99%,Calm,55°F (13°C)


In [119]:
#Connect to Mongo
host_name = "localhost"
port = "27017"

conn_str = {"local" : f"mongodb://{host_name}:{port}/",}
client = pymongo.MongoClient(conn_str["local"])

#Create Database
db = client["Lab5"]

#Create Collections
collection1 = db['7Day_Forecast']
collection2 = db["Current_Weather"]


# 7 Day Forecast into Mongo
weather_forecast.reset_index(inplace=True)
weather_forecast_dict = weather_forecast.to_dict('records')
collection1.insert_many(weather_forecast_dict)

# Current Conditions into Mongo
current_conditions.reset_index(inplace=True)
current_conditions_dict = current_conditions.to_dict('records')
collection2.insert_many(current_conditions_dict)